# Assistants API

The Assistants API currently supports three types of tools: 
*    Code Interpreter, 
*    Retrieval, and 
*    Function calling. 

At a high-level, for a typical assistants API:

1. Create an Assistant in the API by defining its custom instructions and picking a model. 
    * If helpful, enable tools like Code Interpreter, Retrieval, and Function calling.
2. Create a Thread when a user starts a conversation.
3. Add Messages to the Thread as the user ask questions.
4. Run the Assistant on the Thread to trigger responses. 
    * This automatically calls the relevant tools.

Let's create and run an Assistant that uses Code Interpreter.

## Step 1: Create an Assistant

https://platform.openai.com/docs/assistants/overview/step-1-create-an-assistant

In [2]:
from openai import OpenAI
client = OpenAI()

In [4]:
assistant = client.beta.assistants.create(
    name="Math Teacher",
    model="gpt-4-1106-preview",
    instructions="You are a personal math teacher. Write and run code to answer math questions.",
    tools=[{"type": "code_interpreter"}]
)

## Step 2: Create a Thread

**A Thread represents a conversation.** 

We recommend creating one Thread per user, as soon as the user initiates the conversation. Pass any user-specific context and files in this thread by creating Messages.

https://platform.openai.com/docs/assistants/overview/step-2-create-a-thread

In [6]:
thread = client.beta.threads.create()

## Step 3: Add a Message to a Thread

https://platform.openai.com/docs/assistants/overview/step-3-add-a-message-to-a-thread

In [7]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

In [8]:
thread.id

'thread_8bmmGEBduq2Pis2vy2TeWmWT'

In [13]:
client.beta.threads.messages.list(thread.id)

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_mC1JYpADs8ijzxeYZjhDS0Aj', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1700044603, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_8bmmGEBduq2Pis2vy2TeWmWT')], object='list', first_id='msg_mC1JYpADs8ijzxeYZjhDS0Aj', last_id='msg_mC1JYpADs8ijzxeYZjhDS0Aj', has_more=False)

## Step 4: Run the Assistant

Just create a `Run` method that is instantiated from threads, with the given parameters of `thread.id` and `assistant.id`.

In [14]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account."
)

## Step 5: Display the Assistant's Response

This creates a Run in a `queued` status. You can periodically retrieve the Run to check on its status to see if it has moved to `completed`.

In [15]:
run = client.beta.threads.runs.retrieve(
  thread_id=thread.id,
  run_id=run.id
)

In [16]:
run.status

'completed'

In [17]:
messages = client.beta.threads.messages.list(
  thread_id=thread.id
)

In [18]:
messages

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_CyAl7M3AimTuEGH2QmWEK8Cj', assistant_id='asst_ZUoDtY0FsWWZmlONpuZcPQ0a', content=[MessageContentText(text=Text(annotations=[], value='The solution to the equation \\( 3x + 11 = 14 \\) is \\( x = 1 \\).'), type='text')], created_at=1700045245, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_ajJCur3pOmCbMHOPypTlgQFO', thread_id='thread_8bmmGEBduq2Pis2vy2TeWmWT'), ThreadMessage(id='msg_mC1JYpADs8ijzxeYZjhDS0Aj', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I need to solve the equation `3x + 11 = 14`. Can you help me?'), type='text')], created_at=1700044603, file_ids=[], metadata={}, object='thread.message', role='user', run_id=None, thread_id='thread_8bmmGEBduq2Pis2vy2TeWmWT')], object='list', first_id='msg_CyAl7M3AimTuEGH2QmWEK8Cj', last_id='msg_mC1JYpADs8ijzxeYZjhDS0Aj', has_more=False)